**NOTE: This notebook is written for the Google Colab platform. However it can also be run (possibly with minor modifications) as a standard Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import numpy as np
from matplotlib import pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from scipy.stats import norm
from scipy.optimize import minimize
from IPython.display import HTML
from IPython.utils.capture import capture_output
from matplotlib.animation import FuncAnimation

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
from class_utils.download import download_file_maybe_extract
download_file_maybe_extract("https://www.dropbox.com/s/u8u7vcwy3sosbar/titanic.zip?dl=1", directory="data/titanic")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

In [ ]:
#@title -- Auxiliary Functions -- { display-mode: "form" }

# fix the seed of the random generator
# np.random.seed(4)

# GP-based Bayes opti
def random_point(lbound=0, ubound=5):
    return np.random.uniform(0, 5)
    
def next_point(gp, ybest, acquisition_func):
    x = random_point()
    
    best_score = np.inf
    num_restarts = 10
    
    for i in range(num_restarts):
        res = minimize(lambda xx: -acquisition_func(gp,
                           xx.reshape([-1, 1]), ybest).item(),
                       random_point(),
                       method='L-BFGS-B',
                       bounds=[[0, 5]])

        if res.fun < best_score:
            best_score = res.fun
            x = res.x

    return x

# plotting
def plot_distro(
    x, gp, intervals=True, ax=None,
    xlabel="x", ylabel="y", return_preds=False
):
    if ax is None:
        ax = plt.gca()

    y_mean, y_std = gp.predict(x.reshape((-1, 1)), return_std=True) 
    y_mean, y_std = y_mean.reshape(-1), y_std.reshape(-1)
    
    ax.plot(x, y_mean, 'k', lw=3, zorder=9)
    
    if intervals:
        ax.fill_between(x, y_mean - 3*y_std, y_mean + 3*y_std,
                         alpha=0.2, color='b')
    
    ax.set_xlim(np.min(x), np.max(x))
    ax.set_ylim(-5, 5)
    ax.grid(ls='--')
    
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

    if return_preds:
        return y_mean, y_std
    
def plot_distro_func_data(
    x, gp=None, X=None, Y=None, func=None,
    intervals=True, ax=None, return_preds=False
):
    preds = None

    if ax is None:
        ax = plt.gca()

    if not gp is None:
        preds = plot_distro(
            x, gp, intervals=intervals, ax=ax, return_preds=return_preds
        )
        
    if not func is None:
        ax.plot(x, func(x), '--', linewidth=3)
    
    if not X is None and not Y is None:
        ax.scatter(X, Y, c='r', s=50, zorder=10, edgecolors=(0, 0, 0))
    
    return preds

def bayes_opti_anim(fig, gp, x, func, acquisition_func,
                      X=None, Y=None, num_opti_steps=10, ax=None,
                      save_figures=True, save_plain=True,
                      save_plain_nofunc=True, save_nofunc=True):
    with capture_output() as io:
        if ax is None:
            ax = fig.gca()
            
        ax.clear()
        plot_distro_func_data(x, gp, func=func, ax=ax)
        plt.show()
        if save_figures:
            fig.savefig("output/gp_opti_init.svg",
                        bbox_inches="tight", pad_inches=0)
        yield

        if X is None: X = []
        if Y is None: Y = []

        x0 = random_point()
        xbest = x0
        ybest = func(x0)

        for s in range(num_opti_steps):
            nx = next_point(gp, ybest, acquisition_func)
            ny = func(nx)

            if ny < ybest:
                xbest = nx
                ybest = ny

            X.append(nx)
            Y.append(ny)

            gp.fit(np.reshape(X, (-1, 1)),
                   np.reshape(Y, (-1, 1)))
            
            if save_plain:
                ax.clear()
                plot_distro_func_data(x, gp, func=func, X=X, Y=Y, ax=ax,
                                      intervals=False)
                fig.savefig("output/gp_opti_plain_{}.svg".format(s),
                            bbox_inches="tight", pad_inches=0)
                
            if save_plain_nofunc:
                ax.clear()
                plot_distro_func_data(x, gp, X=X, Y=Y, ax=ax,
                                      intervals=False)
                fig.savefig("output/gp_opti_plain_nofunc_{}.svg".format(s),
                            bbox_inches="tight", pad_inches=0)
                
            if save_nofunc:
                ax.clear()
                plot_distro_func_data(x, gp, X=X, Y=Y, ax=ax)
                fig.savefig("output/gp_opti_nofunc_{}.svg".format(s),
                            bbox_inches="tight", pad_inches=0)

            ax.clear()
            plot_distro_func_data(x, gp, func=func, X=X, Y=Y, ax=ax)

            plt.show()
            if save_figures:
                fig.savefig("output/gp_opti_{}.svg".format(s),
                            bbox_inches="tight", pad_inches=0)
            yield

        ax.clear()
        x = np.linspace(0, 5, 100)
        plot_distro_func_data(x, gp, func=func, X=X, Y=Y, ax=ax)
        ax.scatter([xbest], [ybest], c='cyan', marker='x', linewidth=5,
                    s=100, zorder=10, edgecolors=(0, 0, 0))
        plt.show()
        if save_figures:
            fig.savefig("output/gp_opti_final.svg",
                        bbox_inches="tight", pad_inches=0)
        yield
        
    print("solution:", xbest, ybest)
    return

## Bayesian Optimization

### The Intuition behind Bayesian Optimization

There are many different kinds of optimization methods. Some, such as gradient descent are based on gradient information, some even on higher-order derivatives: e.g. the Newton method and its many approximations. Other classes of optimization methods rely on search and employ various clever tricks to make it computationally viable. Yet another class is based on populations of candidate solutions and employs different metaheuristic approaches such as mimicking evolution or other natural phenomena.

But whichever of these methods we are going to choose, there is a class of problems that are exceptionally difficult to solve in a reasonable amount of time: problems where it is extremely computationally expensive to evaluate the objective function. Unfortunately, such problems make up a sizable portion of optimization tasks in practice. They are ubiquitous even within machine learning, because methods have hyperparameters, which usually need to be tuned for the method to achieve optimal performance. But to evaluate a set of hyperparameters we typically need to train the entire machine learning model (which, e.g. in the case of deep learning, can sometimes take days, even weeks) from scratch and to test it.

#### Surrogate Optimization Methods

In cases like that we need to make sure that we are able to squeeze out as much information as possible from every evaluation of the objective function. To that purpose we could use the points at which the objective function has already been evaluated to form a **surrogate model** : a function that we could then optimize instead of the objective function. This is what **surrogate optimization methods**  do.

#### The Bayesian Approach to Surrogate Optimization

There is another question, though. How do we pick the points to evaluate the objective function at, to get the data needed to construct our surrogate model in the first place? We could, perhaps, just take some uniformly random samples from the domain of the objective function and use those.

However, our surrogate model will just be as good as the samples we happened to select. It may still be too expensive to build a sufficiently good surrogate, in which case the optimization process may simply find ways to exploit its errors to get good, but unrealistic scores.

However, there is fortunately a better way: we can refine the surrogate iteratively. We can pick our first point (or a set of points) uniformly, set up our model around it and then use the information to determine which point to pick next. Let us start by setting up a Gaussian process – we'll be trying to minimize the function $\sin([x-2.5]^2)$.



In [ ]:
def func(x):
    return np.sin((x - 2.5) ** 2)

kernel = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 1.0))
gp = GaussianProcessRegressor(kernel=kernel)

x = np.linspace(0, 5, 100)
plot_distro_func_data(x, gp, func=func)

The dashed curve corresponds to our objective function. The thick black line represents the current mean predicted by our GP. As usual, the shaded area corresponds to $\pm3\sigma$ from the mean.

Having received no evidence yet, we only have our prior. We have no reason to think that one region or another is more likely to contain the minimum. So, just as we said, let us select our initial data point uniformly.



In [ ]:
X = [np.random.uniform(0, 5)]
Y = [func(xx) for xx in X]

gp.fit(np.reshape(X, (-1, 1)),
       np.reshape(Y, (-1, 1)))

x = np.linspace(0, 5, 100)
plot_distro_func_data(x, gp, func=func, X=X, Y=Y)

The interesting thing, of course, is that having selected our first point, we now already have some new evidence that we will be able to inform our choice of the next point.

### The Exploration vs. Exploitation Trade-off

When selecting subsequent points to evaluate, we will be trying to balance two subgoals:

* **Exploration:**  Trying to reduce uncertainty in the not yet sufficiently explored regions – to find out whether they are likely to contain the minimum or whether they can be ruled out.
* **Exploitation:**  Trying to focus attention onto areas that we already believe might contain the minimum (i.e. where there is already evidence that the objective function takes low values).
What we have just described is, in fact, known as the **exploration vs. exploitation trade-off**  and is to be encountered in other areas as well, e.g. in reinforcement learning.

The beatiful thing about Bayesian methods such as Gaussian processes is that by maintaining the full posterior instead of just predicting the most likely value, they allow us to explicitly keep track of uncertainty and thus to directly balance exploration and exploitation.

So, all in all, we will on one hand be trying to get as close to the minimum as possible and on the other hand to remove as much uncertainty as possible.

### The Optimization

The component of Bayesian optimization that helps to suggest the next point to sample, is called the **acquisition function** . Given the acquisition function, any off-the-shelf optimization method of choice can be used to find its optimum w.r.t. the surrogate. One of the most popular acquisition functions is the **expected improvement** , which is defined as follows [[jones]](#jones):

$$
EI(\mathbf{x}) \equiv \mathbb{E} \left[ \max(f_{\min} - f(\mathbf{x}), 0) \right].
$$
where $f(\mathbf{x})$ expresses the value of the surrogate at point $\mathbf{x}$ (this is a random variable), hence the need for the expectation; $f_{\min}$ is the best (the lowest) value obtained so far; and the improvement must be non-negative, therefore the $\max$ operator.

Thus, the expected improvement expresses exactly what the name suggests: the expected value of improvement, that is, the difference between the best value so far and the value at $\mathbf{x}$. If the $f(\mathbf{x})$ is Gaussian, i.e.:
$f(\mathbf{x}) \sim \mathcal{N}(\mu(\mathbf{x}), \sigma(\mathbf{x}))$
(which, in the case of Gaussian processes, it, of course, is), the expected improvement can also be expressed in the following more convenient form [[jones]](#jones):

$$
EI(\mathbf{x}) = \left(
    f_{\min} - \mu(\mathbf{x})
\right)\;
\Phi \left(
    \frac{
        f_{\min} - \mu(\mathbf{x})
    }{
        \sigma(\mathbf{x})
    }
\right)
+ \sigma(\mathbf{x}) \phi \left(
    \frac{
        f_{\min} - \mu(\mathbf{x})
    }{
        \sigma(\mathbf{x})
    }
\right)
$$
where $\phi$ is the Gaussian probability density function (PDF) and $\Phi$ is the Gaussian cumulative distribution function (CDF).

To gain more explicit control of the exploration-exploitation trade-off, one can subtract parameter $\xi$ from the difference $f_{\min} - \mu(\mathbf{x})$ [[brochu]](#brochu):

$$
EI(\mathbf{x}) = \left(
    f_{\min} - \mu(\mathbf{x}) - \xi
\right)\;
\Phi \left(
    \frac{
        f_{\min} - \mu(\mathbf{x}) - \xi
    }{
        \sigma(\mathbf{x})
    }
\right)
+ \sigma(\mathbf{x}) \phi \left(
    \frac{
        f_{\min} - \mu(\mathbf{x}) - \xi
    }{
        \sigma(\mathbf{x})
    }
\right)
$$
The difference $f_{\min} - \mu(\mathbf{x})$ is the performance term: it expresses how much better the point $\mathbf{x}$ appears (through the mean of $f(\mathbf{x}$) in comparison to the best value so far. The rest of the criterion is to take variance (uncertainty) into account as well. So by subtracting $\xi$, we are lowering the weight of the performance term, encouraging more exploration.

The value of $\xi = 0.01$, scaled by the signal variance if necessary, has been reported to work well in most cases. Annealing $\xi$ (to encourage exploration at the beginning and expoitation towards the end) has been shown not to work very well empirically [[jones]](#jones).

---
### Task 1: Implement the Expected Improvement

**Implement the expected improvement acquisition function by rewriting the formula shown below into Python code. Use the signature provided in the cell below.** 

Note: you can use `norm.pdf` and `norm.cdf` to compute the Gaussian PDF and CDF respectively.

$$
EI(\mathbf{x}) = \left(
    f_{\min} - \mu(\mathbf{x}) - \xi
\right)\;
\Phi \left(
    \frac{
        f_{\min} - \mu(\mathbf{x}) - \xi
    }{
        \sigma(\mathbf{x})
    }
\right)
+ \sigma(\mathbf{x}) \phi \left(
    \frac{
        f_{\min} - \mu(\mathbf{x}) - \xi
    }{
        \sigma(\mathbf{x})
    }
\right)
$$
---


In [ ]:
def expected_improvement(gp, x, fmin, explo_rate=0.01):
    mu, sigma = gp.predict(x, return_std=True)
    mu = mu.reshape(-1); sigma = sigma.reshape(-1)
    

    
    # ---
    
    
    

Finally, we will use our `expected_improvement` function with a predefined auxiliary function `bayes_opti_anim`, which will take care of optimizing the acquisition function over the surrogate using LBFGS and then iteratively updating the surrogate using the newly suggested points. The results will be presented in the form of an animated figure. We should see how the objective function is gradually explored and the approximate minimum found.



In [ ]:
kernel = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 1.0))
gp = GaussianProcessRegressor(kernel=kernel)

fig = plt.figure()
x = np.linspace(0, 5, 100)
frames = lambda: bayes_opti_anim(fig, gp, x, func=func,
               acquisition_func=expected_improvement,
               num_opti_steps=10)

anim = FuncAnimation(fig, func=lambda x: None,
                     interval=200, frames=frames)
HTML(anim.to_jshtml())

### Visualizing the Expected Improvement

To get more intuition as to what our acquisition function (expected improvement) expresses, let us visualize it for a simple GP conditioned on some data points. The exploration rate is set to $\xi=0.01$.



In [ ]:
X = [0.5, 3.5]
Y = [func(xx) for xx in X]

kernel = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 1.0))
gp = GaussianProcessRegressor(kernel=kernel)
gp.fit(np.reshape(X, (-1, 1)),
       np.reshape(Y, (-1, 1)))

x = np.linspace(0, 5, 100)
fig, axes = plt.subplots(2, 1, sharex=True)

explo_rate=0.01
ac = expected_improvement(gp, x, np.min(Y), explo_rate=explo_rate)

plot_distro_func_data(x, gp, X=X, Y=Y, ax=axes[0])
axes[1].plot(x, ac)
axes[1].set_xlabel("x")
axes[1].set_ylabel("EI(x)")
plt.grid(ls='--')

fig.savefig("output/expected_improvement.svg",
            bbox_inches='tight', pad_inches=0)

### Understanding the Expected Improvement

Now let's try to understand the math behind the expected improvement a bit more:

$$
EI(\mathbf{x}) \equiv \mathbb{E} \left[ \max(f_{\min} - f(\mathbf{x}) - \xi, 0) \right].
$$
Let's go through it all in order.

#### The Actual Improvement

 First, there is the actual improvement expressed as $f_{\min} - f(\mathbf{x})$ in the formula. The larger this difference is, the better $f(\mathbf{x})$ is than the previous best.

#### Bounding the Improvement from Below

The $\max(\text{improvement}, 0)$ puts the lower bound of zero on the improvement. This means that we essentially ignore all cases where $f(\mathbf{x})$ is actually worse than $f_{\min}$ and we default to the improvement of 0 even though the value would otherwise be negative.

As shown in the figure below, this means that we only consider the area **below the dashed red line**  (everything below the dashed line, but the shaded $\pm 3\sigma$ interval is where most of the probablity is). This is the only part that can make a non-zero contribution.



In [ ]:
#@title -- Expected improvement + visualization of clipping explo_rate=0.01 -- { display-mode: "form" }
X = [0.5, 3.5]
Y = [func(xx) for xx in X]

kernel = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 1.0))
gp = GaussianProcessRegressor(kernel=kernel)
gp.fit(np.reshape(X, (-1, 1)),
       np.reshape(Y, (-1, 1)))

x = np.linspace(0, 5, 100)
fig, axes = plt.subplots(2, 1, sharex=True)

explo_rate=0.01
ac = expected_improvement(gp, x, np.min(Y), explo_rate=explo_rate)

y_mean, y_std = plot_distro_func_data(x, gp, X=X, Y=Y, ax=axes[0], return_preds=True)
axes[1].plot(x, ac)
axes[1].set_xlabel("x")
axes[1].set_ylabel("EI(x)")
plt.grid(ls='--')

f_min_xi = np.min(Y) - explo_rate
axes[0].axhline(y=f_min_xi, c='r', ls='--', zorder=10, label="$f_{\min}$")
axes[0].fill_between(x, np.minimum(y_mean - 3*y_std, f_min_xi), f_min_xi,
                     color='r', alpha=0.3, zorder=5, label="unclipped area")
axes[0].legend()

fig.savefig("output/expected_improvement_shaded_001.svg",
            bbox_inches='tight', pad_inches=0)

#### The Expected Value

The rest is simple, really – we take the expected value of improvements below the dashed line, so areas where our confidence interval reaches down towards more optimal values will get preference.

#### The Exploration Rate

To make the effect of exploration rate $\xi$ more obvious, we are going to increase it to $\xi = 0.75$ from $0.01$. As you can see, this shifts the dashed line further down. This aids exploration because points within the margin are ignored, which, of course, benefits regions with more uncertainty, where the $\pm 3 \sigma$ region reaches further down and it is therefore not clipped as much.



In [ ]:
#@title -- Expected improvement + visualization of clipping with explo_rate=0.75 -- { display-mode: "form" }
X = [0.5, 3.5]
Y = [func(xx) for xx in X]

kernel = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 1.0))
gp = GaussianProcessRegressor(kernel=kernel)
gp.fit(np.reshape(X, (-1, 1)),
       np.reshape(Y, (-1, 1)))

x = np.linspace(0, 5, 100)
fig, axes = plt.subplots(2, 1, sharex=True)

explo_rate=0.75
ac = expected_improvement(gp, x, np.min(Y), explo_rate=explo_rate)

y_mean, y_std = plot_distro_func_data(x, gp, X=X, Y=Y, ax=axes[0], return_preds=True)
axes[1].plot(x, ac)
axes[1].set_xlabel("x")
axes[1].set_ylabel("EI(x)")
plt.grid(ls='--')

f_min_xi = np.min(Y) - explo_rate
axes[0].axhline(y=f_min_xi, c='r', ls='--', zorder=10, label=r"$f_{\min} - \xi$")
axes[0].fill_between(x, np.minimum(y_mean - 3*y_std, f_min_xi), f_min_xi,
                     color='r', alpha=0.3, zorder=5, label="unclipped area")
axes[0].legend()

fig.savefig("output/expected_improvement_shaded_075.svg",
            bbox_inches='tight', pad_inches=0)

### References

<a id="jones">[jones]</a> Jones, D.R., Schonlau, M. and Welch, W.J., 1998. Efficient global optimization of expensive black-box functions. Journal of Global optimization, 13(4), pp.455-492. [https://link.springer.com/content/pdf/10.1023/A:1008306431147.pdf](https://link.springer.com/content/pdf/10.1023/A:1008306431147.pdf)

<a id="brochu">[brochu]</a> Brochu, E., Cora, V.M. and De Freitas, N., 2010. A tutorial on Bayesian optimization of expensive cost functions, with application to active user modeling and hierarchical reinforcement learning. arXiv preprint arXiv:1012.2599. [https://arxiv.org/abs/1012.2599](https://arxiv.org/abs/1012.2599)

